In [1]:
! pip install transformers==4.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e78c01a2190567a7ca838c908a23797bf73674130a3d77dd3ff0af77891a7757
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


# **Data**

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.3 MB/s eta 0:00:00


# WASSA dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import pandas as pd
import numpy as np

wassa_train_csv_path = '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/WASSA_train_all.csv'
wassa_dev_csv_path = '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/WASSA_dev_all.csv'

wassa_train = pd.read_csv(wassa_train_csv_path)
wassa_dev = pd.read_csv(wassa_dev_csv_path)


In [5]:
print('wassa_train shape:', wassa_train.shape)
wassa_train['essay'].head(10)

wassa_train shape: (1860, 2)


0    it is really diheartening to read about these ...
1    the phone lines from the suicide prevention li...
2    no matter what your heritage, you should be ab...
3    it is frightening to learn about all these sha...
4    the eldest generation of russians aren't being...
5    middle east is fucked up, I've honestly never ...
6    well first of all whoever wrote this article d...
7    well well well, look at what we have well, the...
8    just another fucked up mental sickness of amer...
9    it seems a horny male college student has fina...
Name: essay, dtype: object

In [6]:
wassa_train['emotion'].head(10)

0    0
1    0
2    6
3    2
4    0
5    3
6    3
7    4
8    3
9    4
Name: emotion, dtype: int64

In [7]:
X_train = wassa_train['essay']
y_train = wassa_train['emotion']

X_test = wassa_dev['essay']
y_test = wassa_dev['emotion']

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1860,)
(1860,)
(270,)
(270,)


In [9]:
# label: 'sadness': 0, 'neutral': 6, 'fear': 2, 'anger': 3, 'disgust': 4, 'surprise': 5, 'joy': 6
y_train.value_counts()

0    647
3    349
6    275
2    194
5    164
4    149
1     82
Name: emotion, dtype: int64

In [10]:
y_test.value_counts()

0    98
3    76
2    31
6    25
1    14
5    14
4    12
Name: emotion, dtype: int64

# **Model**


In [11]:
# Fit and transform X_train using Tfidf Vectorizer with default parameters
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
print('tfidf train shape:', X_train_tfidf.shape)
print('tfidf train type:', X_train_tfidf.dtype)

X_test_tfidf = vectorizer.transform(X_test).toarray()
print('tfidf test:', X_test_tfidf.shape)


tfidf train shape: (1860, 9688)
tfidf train type: float64
tfidf test: (270, 9688)


In [12]:
from sklearn.svm import LinearSVC
clf = LinearSVC().fit(X_train_tfidf, y_train)

from sklearn import metrics
predicted = clf.predict(X_test_tfidf)

acc = metrics.accuracy_score(y_test, predicted)
print('accuracy is: ', acc*100)

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predicted))
print('\n')
print(classification_report(y_test,predicted))

accuracy is:  47.40740740740741
[[76  1  6  7  1  2  5]
 [ 6  0  1  3  1  1  2]
 [10  1 16  0  1  1  2]
 [35  0  0 27  7  2  5]
 [ 5  0  0  4  1  0  2]
 [ 6  0  0  4  1  2  1]
 [ 8  1  1  7  2  0  6]]


              precision    recall  f1-score   support

           0       0.52      0.78      0.62        98
           1       0.00      0.00      0.00        14
           2       0.67      0.52      0.58        31
           3       0.52      0.36      0.42        76
           4       0.07      0.08      0.08        12
           5       0.25      0.14      0.18        14
           6       0.26      0.24      0.25        25

    accuracy                           0.47       270
   macro avg       0.33      0.30      0.31       270
weighted avg       0.45      0.47      0.45       270



# Augmentation using Embedding Word2Vec

In [13]:
! pip install nlpaug==1.1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 KB 25.6 MB/s eta 0:00:00


In [14]:
from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_glove('glove.6B', '.')

In [15]:
# apply augmentation to the train data and save the results into a file
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action
import nlpaug.flow as naf

aug_emb = naf.Sequential([naf.Sometimes([
    naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="substitute")]),
    naf.Sometimes([naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="insert"),
])])


In [16]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [17]:
from tqdm.notebook import tqdm
tqdm.pandas()

data = {'text': X_train, 'label': y_train}
base_train = pd.DataFrame.from_dict(data)
base_train.shape

(1860, 2)

In [18]:
rep = 5 # how many repetitions of the record


emb_train = base_train.copy()
emb_train['paraphrase'] = emb_train['text'].progress_apply(lambda x:aug_emb.augment(x, rep))
emb_train = emb_train.explode('paraphrase').reset_index(drop=True)

  0%|          | 0/1860 [00:00<?, ?it/s]

In [19]:
emb_train.head(20)

,text,label,paraphrase
0,it is really diheartening to read about these ...,0,it is really suzerains diheartening to read ab...
1,it is really diheartening to read about these ...,0,on-line it is langlands really diheartening to...
2,it is really diheartening to read about these ...,0,it is really diheartening to read about these ...
3,it is really diheartening to read about these ...,0,it is really diheartening so read about these ...
4,it is really diheartening to read about these ...,0,madhoo it is really diheartening to read nearl...
5,the phone lines from the suicide prevention li...,0,maiken the phone lines parkmore from shangai t...
6,the phone lines from the suicide prevention li...,0,the phone lines from joss the suicide stealthi...
7,the phone lines from the suicide prevention li...,0,cockettes the phone lines from the confernce s...
8,the phone lines from the suicide prevention li...,0,the subscriber lines from all suicide preventi...
9,the phone lines from the suicide prevention li...,0,the online lines from some suicide prevention ...


# Model with augmented train data (paraphrased data)

In [20]:
# Fit and transform X_train using Tfidf Vectorizer with default parameters
from sklearn.feature_extraction.text import TfidfVectorizer

# !! we should concatenate original train with paraphrased train
new_data = X_train.append(emb_train['paraphrase'])

vectorizer = TfidfVectorizer() #max_features=3000)
X_train_tfidf = vectorizer.fit_transform(new_data).toarray() #emb_train['paraphrase']).toarray()
print('tfidf train shape:', X_train_tfidf.shape)
print('tfidf train type:', X_train_tfidf.dtype)

X_test_tfidf = vectorizer.transform(X_test).toarray()
print('tfidf test:', X_test_tfidf.shape)


tfidf train shape: (11160, 65762)
tfidf train type: float64
tfidf test: (270, 65762)


In [21]:
from sklearn.svm import LinearSVC
# !! we should concatenate original labels with paraphrased text labels
new_labels = y_train.append(emb_train['label'])

clf = LinearSVC().fit(X_train_tfidf, new_labels) # emb_train['label'])

from sklearn import metrics
predicted = clf.predict(X_test_tfidf)

acc = metrics.accuracy_score(y_test, predicted)
print('accuracy is: ', acc*100)

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predicted))
print('\n')
print(classification_report(y_test,predicted))

accuracy is:  45.925925925925924
[[77  1  6  7  0  2  5]
 [ 6  0  2  2  1  1  2]
 [12  0 14  0  1  1  3]
 [32  0  0 27  7  3  7]
 [ 4  0  0  5  2  0  1]
 [ 5  0  0  5  1  2  1]
 [10  1  1  7  2  2  2]]


              precision    recall  f1-score   support

           0       0.53      0.79      0.63        98
           1       0.00      0.00      0.00        14
           2       0.61      0.45      0.52        31
           3       0.51      0.36      0.42        76
           4       0.14      0.17      0.15        12
           5       0.18      0.14      0.16        14
           6       0.10      0.08      0.09        25

    accuracy                           0.46       270
   macro avg       0.30      0.28      0.28       270
weighted avg       0.43      0.46      0.43       270



# Create CSV file from augmented dataset

In [22]:
!cd '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/'
emb_train.to_csv('/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/wassa_all_emb_Aug_5.csv', encoding='utf-8', index=False, sep=',')